In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import datetime

Добавим в переменную `date_list` даты начала и конца периода, за который нам нужны курсы валют.   
Для автоматизации get-запроса, нам нужны даты в формате дд.мм.гггг.

In [2]:
# Для примера возьмём данные за последний месяц.
date_list = list(pd.date_range(start="2023-02-02",end=datetime.datetime.today()))
date_list = [datetime.datetime.strptime(str(i), "%Y-%m-%d 00:00:00").strftime("%d.%m.%Y") for i in date_list]
# возьмём три валюты.
currency_list = ['USD', 'EUR', 'KZT']

In [3]:
def find_currency(currencies, currency_table, date_row) :
    '''
    Функция find_currency является подфункцией в функции ниже currencies_n_dates.
    Передаём из основной функции список всех валют, текст со страницы, лист с будущей строкой в итоговом датафрейме.
    1) проходим циклом по валютам;
    2) в тексте со страницы найдём tr с нужными валютами и составим новую строчку для итогового датафрейма, 
       в которую будут входит одна дата и курсы валют в этот день;
    3) делаем строчку для датафрейма.
    '''
    for currency in currencies:
        found_currency = []
        for tr in currency_table :
            tr_2 = str(tr)
            if currency in tr_2 :
                for td in tr.find_all('td') :
                    found_currency.append(td.text)
                            #По некоторым валютам курс 1 рубля указан на разное количество у.е. (например, 100 тенге),
                            #поэтому курс нужно поделить на это количество.
        date_row.append(float(found_currency[-1].replace(',', '.'))/float(found_currency[2]))
          
    return date_row

In [4]:
def currencies_n_dates(dates, currencies) : 
    '''
    Функция делает итоговый датафрейм с датой и курсами нужных валют.
    1) передаем функции даты и валюты;
    2) каждая строка датафрейма - очередная дата, поэтому проходим циклам по всем датам;
    3) делаем get-запрос к странице курсов валют;
    4) переходим к функции find_currency и делаем строчку для датафрейма;
    5) заполняем и финализируем датафрейм, выводим его как результат функции. 
    '''
    final_df = pd.DataFrame(columns=range(len(currencies)+1))
    for date in dates :
        date_row = [date]     
        #чтобы получить страницу с нужной датой, нужно просто поменять дату в конце
        URL = f'https://www.cbr.ru/currency_base/daily/?UniDbQuery.Posted=True&UniDbQuery.To={date}'
        req = requests.get(URL)
        currency_table = bs(req.text, 'lxml').find_all('tr')
        #переходим к новой функции
        find_currency(currencies, currency_table, date_row)
        #вносим строку по каждой дате
        final_df.loc[len(final_df)] = date_row
    final_df = final_df.set_index(0, drop=True)
    final_df.columns = currencies
    return final_df

Включаем следующую строчку и выводим итоговый датафрейм.

In [5]:
currencies_n_dates(date_list, currency_list)

,USD,EUR,KZT
0,,,
02.02.2023,70.1217,76.2245,0.152270
03.02.2023,70.0414,76.9564,0.151976
04.02.2023,70.3847,76.7344,0.153037
05.02.2023,70.3847,76.7344,0.153037
06.02.2023,70.3847,76.7344,0.153037
07.02.2023,70.5991,76.0347,0.153915
08.02.2023,70.8924,75.9087,0.154888
09.02.2023,71.5763,76.8344,0.156629
10.02.2023,72.8949,78.3223,0.159980
